This experiment notebook is the base for the lake_to_dwh.py file. It will read/load the newspaper htmls, transform them and prepare the features, which are needed for an easy analyse. The analyse will be about the word "klima" and its changing use over time by newspaper. See Readme for more.

In [ ]:
import os
import sys
import logging
sys.path.append(os.path.abspath("pylib"))

import pandas as pd
from handle_sqlite import save_dataframe_to_db, read_table_as_dataframe
from handle_data_processing import process_newspaper_with_context


# Load all the newspapers
Here we will load the csv for one day.

test: saving df as sqlite and read it

In [ ]:
test = {'Name': ['Tom', 'nick', 'chris', 'jack'],
        'Age': [20, 21, 19, 18]}

# Create DataFrame
df = pd.DataFrame(test)
# df (pd.DataFrame): The DataFrame to save.
# table_name (str): The name of the table in the database.
# connection (sqlite3.Connection): SQLite connection object.
# if_exists (str): How to behave if the table already exists. Options are 'fail', 'replace', 'append'.
save_dataframe_to_db(df, "test_table", "data_output/dwh_data.db", "replace")

In [ ]:
read_table_as_dataframe("test_table", "data_output/dwh_data.db")

 orchestrate the processing

In [ ]:
newspapers = [
    {"file_name": "data_input/data_lake/2021-04-02-54books.html", "encoding": "utf-8", "name": "54books", "date": "2021-04-02"},
    {"file_name": "data_input/data_lake/2021-04-02-abendblatt.html", "encoding": "utf-8", "name": "abendblatt", "date": "2021-04-02"},
    {"file_name": "data_input/data_lake/2025-01-14-vice-de.html", "encoding": "utf-8", "name": "vice", "date": "2025-01-24"},
    {"file_name": "data_input/data_lake/2021-04-24-tagesschau.html", "encoding": "utf-8", "name": "tagesschaus", "date": "2021-04-24"},
    {"file_name": "data_input/data_lake/2025-01-24-tagesschau.html", "encoding": "utf-8", "name": "tagesschaus", "date": "2025-01-24"},
]

In [ ]:
metadata_collection = []
context_collection = []

for newspaper in newspapers:
    try:
        metadata, context_data = process_newspaper_with_context(newspaper)
                
        # Add a unique ID for each newspaper in the metadata and add to context
        newspaper_id = len(metadata_collection) + 1  # This can be a simple counter for unique IDs (or use UUID)
        metadata["newspaper_id"] = newspaper_id  # Add newspaper_id to metadata
        
        # Append the metadata to its respective collection
        metadata_collection.append(metadata)
        
        # Append the context data with id to its respective collection if 'klima' was found at least once
        if metadata['klima_mentions_count'] > 0:
            # First add the same newspaper_id to each context data
            for context in context_data:
                context["newspaper_id"] = newspaper_id
            context_collection.extend(context_data) # Using extend here because context_data is already a list of dicts

    except Exception as e:
        logging.error(f"Error processing {newspaper['name']} for {newspaper['date']}: {e}")

In [ ]:
# Convert to DataFrame after processing all newspapers
final_metadata_df = pd.DataFrame(metadata_collection)
final_context_df = pd.DataFrame(context_collection)

In [ ]:
# Save the results to the database
save_dataframe_to_db(final_metadata_df, "newspapers", db_path="data_output/dwh_data.db", if_exists="replace")
save_dataframe_to_db(final_context_df, "context", db_path="data_output/dwh_data.db", if_exists="replace")

In [ ]:
read_table_as_dataframe("newspapers", "data_output/dwh_data.db")

for every collected path open and process the newspaper and append result to list

In [ ]:
final_metadata_df

In [ ]:
final_context_df

take list and append to sqlite.

In [ ]:
final_df

In [ ]:
final_df['pre_context']

In [ ]:
final_df['post_context']